In [70]:
import pandas as pd
import numpy as np
import json
import re

with open('plays.json') as data_file:    
    json_data = json.load(data_file)  


In [71]:
plays_json = pd.json_normalize(json_data)
plays_json.head()

,period,homeScore,scoringPlay,priority,statYardage,awayScore,wallclock,modified,id,text,...,expectedPoints.added,winProbability.before,winProbability.after,winProbability.added,start.shortDownDistanceText,start.possessionText,start.downDistanceText,scoringType.displayName,scoringType.name,scoringType.abbreviation
0,1,0,False,False,0,0,2020-12-21T19:34:38Z,2020-12-21T19:35Z,401256135101849902,Zach Williams kickoff for 60 yds,...,-2.823851,0.000100,0.075747,0.075647,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,False,False,-2,0,2020-12-21T19:36:09Z,2020-12-21T19:36Z,401256135101855901,Camerun Peoples run for a loss of 2 yards to t...,...,-1.372518,0.918380,0.898592,-0.019788,1st & 10,APP 35,1st & 10 at APP 35,NaN,NaN,NaN
2,1,0,False,False,0,0,2020-12-21T19:36:29Z,2020-12-21T19:36Z,401256135101858101,Zac Thomas pass incomplete to Jalen Virgil,...,-0.958074,0.898592,0.883065,-0.015527,2nd & 12,APP 33,2nd & 12 at APP 33,NaN,NaN,NaN
3,1,0,False,False,4,0,2020-12-21T19:37:31Z,2020-12-21T19:37Z,401256135101864601,Zac Thomas run for 4 yds to the AppSt 37,...,-0.664023,0.883065,0.879816,-0.003248,3rd & 12,APP 33,3rd & 12 at APP 33,NaN,NaN,NaN
4,1,0,False,False,0,0,2020-12-21T19:38:31Z,2020-12-21T19:38Z,401256135101867201,"Xavier Subotsch punt for 29 yds, downed at the...",...,-0.878193,0.879816,0.880597,0.000781,4th & 8,APP 37,4th & 8 at APP 37,NaN,NaN,NaN


In [72]:
plays_json.columns

Index(['period', 'homeScore', 'scoringPlay', 'priority', 'statYardage',
       'awayScore', 'wallclock', 'modified', 'id', 'text', 'playType',
       'start.distance', 'start.yardLine', 'start.team.id', 'start.down',
       'start.yardsToEndzone', 'start.posTeamTimeouts',
       'start.defTeamTimeouts', 'clock.displayValue', 'type.id', 'type.text',
       'type.abbreviation', 'end.shortDownDistanceText', 'end.possessionText',
       'end.downDistanceText', 'end.distance', 'end.yardLine', 'end.team.id',
       'end.down', 'end.yardsToEndzone', 'end.posTeamTimeouts',
       'end.defTeamTimeouts', 'expectedPoints.before', 'expectedPoints.after',
       'expectedPoints.added', 'winProbability.before', 'winProbability.after',
       'winProbability.added', 'start.shortDownDistanceText',
       'start.possessionText', 'start.downDistanceText',
       'scoringType.displayName', 'scoringType.name',
       'scoringType.abbreviation'],
      dtype='object')

In [195]:
##--Play type vectors------
scores_vec = [
"Blocked Punt Touchdown",
"Blocked Punt (Safety)",
"Punt (Safety)",
"Blocked Field Goal Touchdown",
"Missed Field Goal Return Touchdown",
"Fumble Recovery (Opponent) Touchdown",
"Fumble Return Touchdown",
"Interception Return Touchdown",
"Pass Interception Return Touchdown",
"Punt Touchdown",
"Punt Return Touchdown",
"Sack Touchdown",
"Uncategorized Touchdown",
"Defensive 2pt Conversion",
"Uncategorized",
"Two Point Rush",
"Safety",
"Penalty (Safety)",
"Punt Team Fumble Recovery Touchdown",
"Kickoff Team Fumble Recovery Touchdown",
"Kickoff (Safety)",
"Passing Touchdown",
"Rushing Touchdown",
"Field Goal Good",
"Pass Reception Touchdown",
"Fumble Recovery (Own) Touchdown"
]

defense_score_vec = [
"Blocked Punt Touchdown",
"Blocked Field Goal Touchdown",
"Missed Field Goal Return Touchdown",
"Punt Return Touchdown",
"Fumble Recovery (Opponent) Touchdown",
"Fumble Return Touchdown",
"Kickoff Return Touchdown",
"Defensive 2pt Conversion",
"Safety",
"Sack Touchdown",
"Interception Return Touchdown",
"Pass Interception Return Touchdown",
"Uncategorized Touchdown"
]
turnover_vec = [
  "Blocked Field Goal",
"Blocked Field Goal Touchdown",
"Blocked Punt",
"Blocked Punt Touchdown",
"Field Goal Missed",
"Missed Field Goal Return",
"Missed Field Goal Return Touchdown",
"Fumble Recovery (Opponent)",
"Fumble Recovery (Opponent) Touchdown",
"Fumble Return Touchdown",
"Defensive 2pt Conversion",
"Interception",
"Interception Return",
"Interception Return Touchdown",
"Pass Interception Return",
"Pass Interception Return Touchdown",
"Kickoff Team Fumble Recovery",
"Kickoff Team Fumble Recovery Touchdown",
"Punt Touchdown",
"Punt Return Touchdown",
"Sack Touchdown",
"Uncategorized Touchdown"
]
normalplay = [
"Rush",
"Pass",
"Pass Reception",
"Pass Incompletion",
"Pass Completion",
"Sack",
"Fumble Recovery (Own)"
]
penalty = [
'Penalty', 
'Penalty (Kickoff)',
'Penalty (Safety)'
]
offense_score_vec = [
"Passing Touchdown",
"Rushing Touchdown",
"Field Goal Good",
"Pass Reception Touchdown",
"Fumble Recovery (Own) Touchdown",
"Punt Touchdown", #<--- Punting Team recovers the return team fumble and scores
"Punt Team Fumble Recovery Touchdown",
"Kickoff Touchdown", #<--- Kickoff Team recovers the return team fumble and scores
"Kickoff Team Fumble Recovery Touchdown"
]
punt_vec = [
"Blocked Punt",
"Blocked Punt Touchdown",
"Blocked Punt (Safety)",
"Punt (Safety)",
"Punt",
"Punt Touchdown",
"Punt Team Fumble Recovery",
"Punt Team Fumble Recovery Touchdown",
"Punt Return Touchdown"
]
kickoff_vec = [
"Kickoff",
"Kickoff Return (Offense)",
"Kickoff Return Touchdown",
"Kickoff Touchdown",
"Kickoff Team Fumble Recovery",
"Kickoff Team Fumble Recovery Touchdown",
"Kickoff (Safety)",
"Penalty (Kickoff)"
]
int_vec = [
"Interception",
"Interception Return",
"Interception Return Touchdown",
"Pass Interception",
"Pass Interception Return",
"Pass Interception Return Touchdown"
]

def clean_pbp_dat(play_df):
    play_df.id = play_df.id.astype(int)
    play_df = play_df.sort_values(by="id", ascending=True)
    #-- Touchdowns----
    play_df["scoring_play"] = np.where(play_df["type.text"].isin(scores_vec), 1, 0)
    play_df["td_play"] = play_df.text.str.match("touchdown|for a TD", case=False, flags=0, na=False)
    play_df["touchdown"] = play_df["type.text"].str.match("touchdown", case=False, flags=0, na=False)
    play_df["safety"] = play_df["text"].str.match("safety", case=False, flags=0, na=False)

#       #-- Fumbles----
    play_df["fumble_vec"] = play_df["text"].str.match("fumble", case=False, flags=0, na=False)
    
#       #-- Kicks----
    play_df["kickoff_play"] = np.where(play_df["type.text"].isin(kickoff_vec), 1, 0)
    play_df["kickoff_tb"] = np.where(play_df["text"].str.match("touchback", case=False, flags=0, na=False) & play_df.kickoff_play == True, 1, 0)
    play_df["kickoff_onside"] = np.where(play_df["text"].str.match("on-side|onside|on side", case=False, flags=0, na=False) & play_df.kickoff_play == True, 1, 0)
    play_df["kickoff_oob"] = np.where(play_df["text"].str.match("out-of-bounds|out of bounds", case=False, flags=0, na=False) & play_df.kickoff_play == True, 1, 0)
    play_df["kickoff_fair_catch"] = np.where(play_df["text"].str.match("fair catch|fair caught", case=False, flags=0, na=False) & play_df.kickoff_play == True, 1, 0)
    play_df["kickoff_downed"] = np.where(play_df["text"].str.match("downed", case=False, flags=0, na=False) & play_df.kickoff_play == True, 1, 0)
    play_df["kick_play"] = play_df["text"].str.match("kick|kickoff", case=False, flags=0, na=False)
    play_df["kickoff_safety"] = np.where(~play_df["type.text"].isin(['Blocked Punt','Penalty']) & play_df["text"].str.match("kickoff", case=False, flags=0, na=False) & play_df.safety == True, 1, 0)
    
#       #-- Punts----
    play_df["punt"] = np.where(play_df["type.text"].isin(punt_vec), 1, 0)
    play_df["punt_play"] = play_df["text"].str.match("punt", case=False, flags=0, na=False)
    play_df["punt_tb"] = np.where(play_df["text"].str.match("touchback", case=False, flags=0, na=False) & play_df.punt == True, 1, 0)
    play_df["punt_oob"] = np.where(play_df["text"].str.match("out-of-bounds|out of bounds", case=False, flags=0, na=False) & play_df.punt == True, 1, 0)
    play_df["punt_fair_catch"] = np.where(play_df["text"].str.match("fair catch|fair caught", case=False, flags=0, na=False) & play_df.punt == True, 1, 0)
    play_df["punt_downed"] = np.where(play_df["text"].str.match("downed", case=False, flags=0, na=False) & play_df.punt == True, 1, 0)
    play_df["punt_safety"] = np.where(play_df["type.text"].isin(['Blocked Punt','Punt']) & play_df["text"].str.match("punt", case=False, flags=0, na=False) & play_df.safety == True, 1, 0)
    play_df["penalty_safety"] = np.where(play_df["type.text"].isin(['Penalty']) & play_df.safety == True, 1, 0)
    play_df["punt_blocked"] = np.where(play_df["text"].str.match("blocked", case=False, flags=0, na=False) & play_df.punt == True, 1, 0)

#       #-- Pass/Rush----
    play_df['rush'] = np.where(
        ((play_df["type.text"] == "Rush")
        | (play_df["type.text"] == "Rushing Touchdown")
        | (play_df["type.text"].isin(["Safety","Fumble Recovery (Opponent)","Fumble Recovery (Opponent) Touchdown", "Fumble Recovery (Own)", "Fumble Recovery (Own) Touchdown", "Fumble Return Touchdown"]) & play_df["text"].str.match("run for", case=False, flags=0, na=False))),
        1, 0
    )
    
    play_df['pass'] = np.where(
        (
        (play_df["type.text"].isin(["Pass Reception", "Pass Completion","Pass Touchdown","Sack","Pass","Inteception","Pass Interception Return", "Interception Return Touchdown","Pass Incompletion","Sack Touchdown"]))
        | ((play_df["type.text"] == "Safety") & play_df["text"].str.match("sacked", case=False, flags=0, na=False))
        | ((play_df["type.text"] == "Safety") & play_df["text"].str.match("pass complete", case=False, flags=0, na=False))
         | ((play_df["type.text"] == "Fumble Recovery (Own)") & play_df["text"].str.match("pass complete|pass incomplete|pass intercepted", case=False, flags=0, na=False))
         | ((play_df["type.text"] == "Fumble Recovery (Own)") & play_df["text"].str.match("sacked", case=False, flags=0, na=False))
         | ((play_df["type.text"] == "Fumble Recovery (Own) Touchdown") & play_df["text"].str.match("pass complete|pass incomplete|pass intercepted", case=False, flags=0, na=False))
          | ((play_df["type.text"] == "Fumble Recovery (Own) Touchdown") & play_df["text"].str.match("sacked", case=False, flags=0, na=False))
          | ((play_df["type.text"] == "Fumble Recovery (Opponent)") & play_df["text"].str.match("pass complete|pass incomplete|pass intercepted", case=False, flags=0, na=False))
           | ((play_df["type.text"] == "Fumble Recovery (Opponent)") & play_df["text"].str.match("sacked", case=False, flags=0, na=False))
           | ((play_df["type.text"] == "Fumble Recovery (Opponent) Touchdown") & play_df["text"].str.match("pass complete|pass incomplete", case=False, flags=0, na=False))
           | ((play_df["type.text"] == "Fumble Return Touchdown") & play_df["text"].str.match("pass complete|pass incomplete", case=False, flags=0, na=False))
            | ((play_df["type.text"] == "Fumble Return Touchdown") & play_df["text"].str.match("sacked", case=False, flags=0, na=False))
        ),
        1, 0
    )

      #-- Sacks----
    play_df['sack_vec'] = np.where(
        (
            (play_df["type.text"].isin(["Sack", "Sack Touchdown"]))
            | ((play_df["type.text"].isin(["Fumble Recovery (Own)", "Fumble Recovery (Own) Touchdown", "Fumble Recovery (Opponent)", "Fumble Recovery (Opponent) Touchdown", "Fumble Return Touchdown"]) 
               & (play_df["pass"] == 1)
               & (play_df["text"].str.match("sacked", case=False, flags=0, na=False))
              ))
        ), 1, 0)
    
    play_df['type.text'] = np.where(play_df["text"].str.match(" coin toss ", case=False, flags=0, na=False), "Coin Toss", play_df["type.text"])
    play_df['change_of_poss'] = np.where(play_df["start.team.id"] == play_df["start.team.id"].shift(-1), 0, 1)
    play_df['change_of_poss'] = np.where(play_df['change_of_poss'].isna(), 0, play_df['change_of_poss'])

      ## Fix Strip-Sacks to Fumbles----
    play_df['type.text'] = np.where(
        (play_df.fumble_vec == 1) 
            & (play_df["pass"] == 1) 
            & (play_df.change_of_poss == 1) 
            & (play_df.td_play == 0) 
            & (play_df["start.down"] != 4) 
            & ~(play_df['type.text'].isin(defense_score_vec)),
        "Fumble Recovery (Opponent)", play_df['type.text']
    )
    
    play_df['type.text'] = np.where(
        (play_df.fumble_vec == 1) 
            & (play_df["pass"] == 1) 
            & (play_df.change_of_poss == 1) 
            & (play_df.td_play == 1),
        "Fumble Recovery (Opponent) Touchdown", play_df['type.text']
    )
  #       ## Fix rushes with fumbles and a change of possession to fumbles----  
    play_df['type.text'] = np.where(
        (play_df.fumble_vec == 1) 
            & (play_df["rush"] == 1) 
            & (play_df.change_of_poss == 1) 
            & (play_df.td_play == 0) 
            & (play_df["start.down"] != 4) 
            & ~(play_df['type.text'].isin(defense_score_vec)),
        "Fumble Recovery (Opponent)", play_df['type.text']
    )

    play_df['type.text'] = np.where(
        (play_df.fumble_vec == 1) 
            & (play_df["rush"] == 1) 
            & (play_df.change_of_poss == 1) 
            & (play_df.td_play == 1),
        "Fumble Recovery (Opponent) Touchdown", play_df['type.text']
    )

#       ## Portion of touchdown check for plays where touchdown is not listed in the play_type--
    play_df["td_check"] = play_df["text"].str.contains("Touchdown")

#       #-- Fix kickoff fumble return TDs ----
    play_df['type.text'] = np.where(
            (play_df.kickoff_play == 1) 
            & (play_df.fumble_vec == 1) 
            & (play_df.td_play == 1)
            & (play_df.td_check == 1),
        f"{play_df['type.text']} Touchdown", play_df['type.text']
    )

#       #-- Fix punt return TDs ----
    play_df['type.text'] = np.where(
            (play_df.punt_play == 1) 
            & (play_df.td_play == 1)
            & (play_df.td_check == 1),
        f"{play_df['type.text']} Touchdown", play_df['type.text']
    )

#       #-- Fix kick return TDs----
    play_df['type.text'] = np.where(
            (play_df.kickoff_play == 1) 
            & (play_df.fumble_vec == 0) 
            & (play_df.td_play == 1)
            & (play_df.td_check == 1),
        f"Kickoff Return Touchdown", play_df['type.text']
    )
    
#       #-- Fix rush/pass tds that aren't explicit----
    play_df['type.text'] = np.where(
            (play_df.td_play == 1) 
            & (play_df.rush == 1) 
            & (play_df.fumble_vec == 0) 
            & (play_df.td_check == 1),
        f"Rushing Touchdown", play_df['type.text']
    )
    
    play_df['type.text'] = np.where(
            (play_df.td_play == 1) 
            & (play_df["pass"] == 1) 
            & (play_df.fumble_vec == 0) 
            & (play_df.td_check == 1)
            & ~(play_df['type.text'].isin(int_vec)),
        f"Passing Touchdown", play_df['type.text']
    )
    
    play_df['type.text'] = np.where(
            (play_df["pass"] == 1) 
            & (play_df['type.text'].isin(["Pass Reception", "Pass Completion", "Pass"]))
            & (play_df.statYardage == play_df["start.yardsToEndzone"])
            & (play_df.fumble_vec == 0)
            & ~(play_df['type.text'].isin(int_vec)),
        f"Passing Touchdown", play_df['type.text']
    )
    
    play_df['type.text'] = np.where(
            (play_df['type.text'].isin(["Blocked Field Goal"])) 
            & (play_df['text'].str.match("for a TD", case=False, flags=0, na=False)),
        f"Blocked Field Goal Touchdown", play_df['type.text']
    )


#       #-- Fix duplicated TD play_type labels----
    play_df['type.text'] = np.where(play_df['type.text'] == "Punt Touchdown Touchdown", "Punt Touchdown", play_df['type.text'])
    play_df['type.text'] = np.where(play_df['type.text'] == "Fumble Return Touchdown Touchdown", "Fumble Return Touchdown", play_df['type.text'])
    play_df['type.text'] = np.where(play_df['type.text'] == "Rushing Touchdown Touchdown", "Rushing Touchdown", play_df['type.text'])
    play_df['type.text'] = np.where(play_df['type.text'] == "Uncategorized Touchdown Touchdown", "Uncategorized Touchdown", play_df['type.text'])

#       #-- Fix Pass Interception Return TD play_type labels----
    play_df['type.text'] = np.where(play_df["text"].str.match("pass intercepted for a TD", case=False, flags=0, na=False), "Interception Return Touchdown", play_df["type.text"])

#       #-- Fix Sack/Fumbles Touchdown play_type labels----
    play_df['type.text'] = np.where(
        play_df["text"].str.match("sacked", case=False, flags=0, na=False)
        & play_df["text"].str.match("fumbled", case=False, flags=0, na=False)
        & play_df["text"].str.match("TD", case=False, flags=0, na=False),
        "Fumble Recovery (Opponent) Touchdown", play_df["type.text"]
    )

#       #-- Fix generic pass plays ----
#       ##-- first one looks for complete pass
    play_df['type.text'] = np.where((play_df['type.text'] == "Pass") & play_df.text.str.contains("pass complete"),
                         "Pass Completion", play_df['type.text'])

#       ##-- second one looks for incomplete pass
    play_df['type.text'] = np.where((play_df['type.text'] == "Pass") & play_df.text.str.contains("pass incomplete"),
                         "Pass Incompletion", play_df['type.text'])

#       ##-- third one looks for interceptions 
    play_df['type.text'] = np.where((play_df['type.text'] == "Pass") & play_df.text.str.contains("pass intercepted"),
                         "Pass Interception", play_df['type.text'])

#       ##-- fourth one looks for sacked
    play_df['type.text'] = np.where((play_df['type.text'] == "Pass") & play_df.text.str.contains("sacked"), "Sack", play_df['type.text'])

#       ##-- fifth one play type is Passing Touchdown, but its intercepted 
    play_df['type.text'] = np.where((play_df['type.text'] == "Passing Touchdown") & play_df.text.str.contains("pass intercepted for a TD"),
                         "Interception Return Touchdown", play_df['type.text'])

#       #--- Moving non-Touchdown pass interceptions to one play_type: "Interception Return" -----
    play_df['type.text'] = np.where(play_df['type.text'].isin(["Interception"]), "Interception Return", play_df['type.text'])
    play_df['type.text'] = np.where(play_df['type.text'].isin(["Pass Interception"]), "Interception Return", play_df['type.text'])
    play_df['type.text'] = np.where(play_df['type.text'].isin(["Pass Interception Return"]), "Interception Return", play_df['type.text'])

#       #--- Moving Kickoff/Punt Touchdowns without fumbles to Kickoff/Punt Return Touchdown
    play_df['type.text'] = np.where((play_df['type.text'] == "Kickoff Touchdown") & (play_df.fumble_vec == 0),  "Kickoff Return Touchdown", play_df['type.text'])

    play_df['type.text'] = np.where(play_df['type.text'].isin(["Kickoff", "Kickoff Return (Offense)"]) & 
                           (play_df.fumble_vec == 1) & (play_df.change_of_poss == 1), 
                         "Kickoff Team Fumble Recovery", play_df['type.text'])

    play_df['type.text'] = np.where((play_df['type.text'] == "Punt Touchdown") & (play_df.fumble_vec == 0), "Punt Return Touchdown", play_df['type.text'])
    play_df['type.text'] = np.where((play_df['type.text'] == "Punt") & (play_df.fumble_vec == 1) & (play_df.change_of_poss == 0), "Punt Team Fumble Recovery", play_df['type.text'])

    play_df['type.text'] = np.where(play_df['type.text'].isin(["Punt Touchdown"]), "Punt Team Fumble Recovery Touchdown", play_df['type.text'])
    play_df['type.text'] = np.where(play_df['type.text'].isin(["Kickoff Touchdown"]), "Kickoff Team Fumble Recovery Touchdown", play_df['type.text'])
    play_df['type.text'] = np.where((play_df['type.text'].isin(["Fumble Return Touchdown"])) & ((play_df["pass"] == 1) | (play_df["rush"] == 1)), "Fumble Recovery (Opponent) Touchdown", play_df['type.text'])

#       #--- Safeties (kickoff, punt, penalty) ----
    play_df['type.text'] = np.where(
        (play_df['type.text'].isin(["Pass Reception", "Rush", "Rushing Touchdown"]) 
         & ((play_df["pass"] == 1) | (play_df["rush"] == 1)) 
         & (play_df["safety"] == True))
        , "Safety", play_df["type.text"]
    )
    
    play_df['type.text'] = np.where(
            (play_df.kickoff_safety == 1),
        f"Kickoff (Safety)", play_df['type.text']
    )
    
    play_df['type.text'] = np.where(
            (play_df.punt_safety == 1)
            | (play_df.penalty_safety == 1),
        f"{play_df['type.text']} (Safety)", play_df['type.text']
    )


#     #-- 'Penalty' in play text ----

#       #-- T/F flag conditions penalty_flag 
    play_df['penalty_flag'] = False
    play_df.loc[(play_df['type.text'] == "penalty"), 'penalty_flag'] = True
    play_df.loc[play_df["text"].str.match("penalty", case=False, flags=0, na=False), 'penalty_flag'] = True

#       #-- T/F flag conditions penalty_declined 
    play_df['penalty_declined'] = False
    play_df.loc[(play_df['type.text'] == "penalty") & play_df["text"].str.match("declined", case=False, flags=0, na=False), 'penalty_declined'] = True
    play_df.loc[play_df["text"].str.match("penalty", case=False, flags=0, na=False) & play_df["text"].str.match("declined", case=False, flags=0, na=False), 'penalty_declined'] = True

#       #-- T/F flag conditions penalty_no_play 
    play_df['penalty_no_play'] = False
    play_df.loc[(play_df['type.text'] == "penalty") & play_df["text"].str.match("no play", case=False, flags=0, na=False), 'penalty_no_play'] = True
    play_df.loc[play_df["text"].str.match("penalty", case=False, flags=0, na=False) & play_df["text"].str.match("no play", case=False, flags=0, na=False), 'penalty_no_play'] = True

#       #-- T/F flag conditions penalty_offset 
    play_df['penalty_offset'] = False
    play_df.loc[(play_df['type.text'] == "penalty") & play_df["text"].str.match("off-setting", case=False, flags=0, na=False), 'penalty_offset'] = True
    play_df.loc[play_df["text"].str.match("penalty", case=False, flags=0, na=False) & play_df["text"].str.match("off-setting", case=False, flags=0, na=False), 'penalty_offset'] = True

#       #-- T/F flag conditions penalty_1st_conv 
    play_df['penalty_1st_conv'] = False
    play_df.loc[(play_df['type.text'] == "penalty") & play_df["text"].str.match("1st down", case=False, flags=0, na=False), 'penalty_1st_conv'] = True
    play_df.loc[play_df["text"].str.match("penalty", case=False, flags=0, na=False) & play_df["text"].str.match("1st down", case=False, flags=0, na=False), 'penalty_1st_conv'] = True

#       #-- T/F flag for penalty text but not penalty play type -- 
    play_df['penalty_text'] = False
    play_df.loc[play_df["text"].str.match("penalty", case=False, flags=0, na=False) & ~(play_df['type.text'] == "penalty") & ~play_df["text"].str.match("declined", case=False, flags=0, na=False) & ~play_df["text"].str.match("off-setting", case=False, flags=0, na=False) & ~play_df["text"].str.match("no play", case=False, flags=0, na=False), 'penalty_text'] = True

#       #--- Sacks ----
    play_df['sack'] = ((play_df['type.text'].isin(["Sack"]) 
                           | (play_df['type.text'].isin(["Fumble Recovery (Own)", "Fumble Recovery (Own) Touchdown", "Fumble Recovery (Opponent)", "Fumble Recovery (Opponent) Touchdown"]) & play_df['pass'] == 1 & play_df["text"].str.match("sacked", case=False, flags=0, na=False)))
                           | (play_df['type.text'].isin(["Safety"]) & play_df["text"].str.match("sacked", case=False, flags=0, na=False)))

#       #--- Interceptions ------
    play_df["int"] = play_df["type.text"].isin(["Interception Return", "Interception Return Touchdown"])
    play_df["int_td"] = play_df["type.text"].isin(["Interception Return Touchdown"])

#       #--- Pass Completions, Attempts and Targets -------
    play_df['completion'] = ((play_df['type.text'].isin(["Pass Reception", "Pass Completion", "Passing Touchdown"])) 
                           | (play_df['type.text'].isin(["Fumble Recovery (Own)", "Fumble Recovery (Own) Touchdown", "Fumble Recovery (Opponent)", "Fumble Recovery (Opponent) Touchdown"]) & play_df['pass'] == 1 & ~play_df["text"].str.match("sacked", case=False, flags=0, na=False)))

    play_df['pass_attempt'] = ((play_df['type.text'].isin(["Pass Reception", "Pass Completion", "Passing Touchdown", "Pass Incompletion"])) 
                           | (play_df['type.text'].isin(["Fumble Recovery (Own)", "Fumble Recovery (Own) Touchdown", "Fumble Recovery (Opponent)", "Fumble Recovery (Opponent) Touchdown"]) & play_df['pass'] == 1 & ~play_df["text"].str.match("sacked", case=False, flags=0, na=False)))

    play_df['target'] = ((play_df['type.text'].isin(["Pass Reception", "Pass Completion", "Passing Touchdown", "Pass Incompletion"])) 
                           | (play_df['type.text'].isin(["Fumble Recovery (Own)", "Fumble Recovery (Own) Touchdown", "Fumble Recovery (Opponent)", "Fumble Recovery (Opponent) Touchdown"]) & play_df['pass'] == 1 & ~play_df["text"].str.match("sacked", case=False, flags=0, na=False)))

#       #--- Pass/Rush TDs ------
    play_df['pass_td'] = (play_df["type.text"] == "Passing Touchdown")
    play_df['rush_td'] = (play_df["type.text"] == "Rushing Touchdown")
    
#       #-- Change of possession via turnover
    play_df['turnover_vec'] = play_df["type.text"].isin(turnover_vec)
    play_df['offense_score_play'] = play_df["type.text"].isin(offense_score_vec)
    play_df['defense_score_play'] = play_df["type.text"].isin(defense_score_vec)
    play_df['downs_turnover'] = np.where(
      (play_df["type.text"].isin(normalplay))
      & (play_df["statYardage"] < play_df["start.distance"])
      & (play_df["start.down"] == 4)
      & (play_df["penalty_1st_conv"] != 1)
    , 1, 0)

#       #-- Touchdowns----
    play_df['scoring_play'] = play_df["type.text"].isin(scores_vec)
    play_df['yds_punted'] = play_df["text"].str.extract("(?<= punt for)[^,]+(\d+)", flags=re.IGNORECASE).astype(float)
    play_df['yds_punt_gained'] = np.where(play_df.punt == 1, play_df["statYardage"], None)
    play_df['fg_inds'] = play_df["type.text"].str.match("Field Goal", case=False, flags=0, na=False)
    play_df['fg_made'] = (play_df["type.text"] == "Field Goal Good")
    play_df['yds_fg'] = play_df["text"].str.extract("(\d{0,2}) Yd|Yard FG|Field", flags=re.IGNORECASE).astype(float)
    play_df['start.yardsToEndzone'] = np.where(play_df['fg_inds'] == True, play_df['yds_fg'] - 17, play_df["start.yardsToEndzone"])

    play_df["pos_unit"] = np.select(
        [
            play_df.punt == 1, 
            play_df.kickoff_play == 1,
            play_df.fg_inds == True,
            play_df["type.text"] == "Defensive 2pt Conversion"
        ], 
        [
            'Punt Offense', 
            'Kickoff Return', 
            'Field Goal Offense',
            'Offense'
        ], 
        default='Offense'
    )
    
    play_df["def_pos_unit"] = np.select(
        [
            play_df.punt == 1, 
            play_df.kickoff_play == 1,
            play_df.fg_inds == True,
            play_df["type.text"] == "Defensive 2pt Conversion"
        ], 
        [
            'Punt Return', 
            'Kickoff Defense', 
            'Field Goal Defense',
            'Defense'
        ], 
        default='Defense'
    )
#       #--- Lags/Leads play type ----
    play_df['lag_play_type3'] = play_df['type.text'].shift(3)
    play_df['lag_play_type2'] = play_df['type.text'].shift(2)
    play_df['lag_play_type'] = play_df['type.text'].shift(1)
    
    play_df['lead_play_type'] = play_df['type.text'].shift(-1)
    play_df['lead_play_type2'] = play_df['type.text'].shift(-2)
    play_df['lead_play_type3'] = play_df['type.text'].shift(-3)

    return play_df

In [196]:
plays_json = clean_pbp_dat(plays_json)
plays_json

,period,homeScore,scoringPlay,priority,statYardage,awayScore,wallclock,modified,id,text,...,lag_play_type,lead_play_type,lead_play_type2,lead_play_type3,penalty_flag,penalty_declined,penalty_no_play,penalty_offset,penalty_1st_conv,penalty_text
0,1,0,False,False,0,0,2020-12-21T19:34:38Z,2020-12-21T19:35Z,401256135101849902,Zach Williams kickoff for 60 yds,...,NaN,Rush,Pass Incompletion,Rush,False,False,False,False,False,False
1,1,0,False,False,-2,0,2020-12-21T19:36:09Z,2020-12-21T19:36Z,401256135101855901,Camerun Peoples run for a loss of 2 yards to t...,...,Kickoff,Pass Incompletion,Rush,Punt,False,False,False,False,False,False
2,1,0,False,False,0,0,2020-12-21T19:36:29Z,2020-12-21T19:36Z,401256135101858101,Zac Thomas pass incomplete to Jalen Virgil,...,Rush,Rush,Punt,Rush,False,False,False,False,False,False
3,1,0,False,False,4,0,2020-12-21T19:37:31Z,2020-12-21T19:37Z,401256135101864601,Zac Thomas run for 4 yds to the AppSt 37,...,Pass Incompletion,Punt,Rush,Rush,False,False,False,False,False,False
4,1,0,False,False,0,0,2020-12-21T19:38:31Z,2020-12-21T19:38Z,401256135101867201,"Xavier Subotsch punt for 29 yds, downed at the...",...,Rush,Rush,Rush,Pass Incompletion,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,4,49,False,False,16,23,2020-12-23T03:23:40Z,2020-12-23T03:23Z,401256137104979401,Zach Wilson pass complete to Dax Milne for 16 ...,...,Pass Incompletion,Timeout,Rush,Rush,False,False,False,False,False,False
375,4,49,False,False,0,23,2020-12-23T03:24:04Z,2020-12-23T03:24Z,401256137104986501,"Timeout UCF, clock 01:34",...,Pass Reception,Rush,Rush,Rush,False,False,False,False,False,False
376,4,49,False,False,-2,23,2020-12-23T03:28:13Z,2020-12-23T03:28Z,401256137104986901,TEAM run for a loss of 2 yards to the UCF 6,...,Timeout,Rush,Rush,NaN,False,False,False,False,False,False
377,4,49,False,False,-1,23,2020-12-23T03:28:45Z,2020-12-23T03:28Z,401256137104994401,TEAM run for a loss of 1 yard to the UCF 7,...,Rush,Rush,NaN,NaN,False,False,False,False,False,False
